In [9]:
import pandas

labels = ["id", "survived", "class", "sex", "age", "sibling/spouse", "parent/children"]
data = pandas.read_csv("./train.modified.csv")

for index, value in data["Sex"].items():
    if value == "male":
        data.loc[index, "Sex"] = 0
    else:
        data.loc[index, "Sex"] = 1

data = data.fillna(0)

data["Age"] = (data["Age"] - data["Age"].min()) / (data["Age"].max() - data["Age"].min())

data.to_csv("./train.processed.csv", index=False)

# thanks chatGPT for helping me write this

In [17]:
import pandas
from torch import tensor, float32
labels = ["id", "survived", "class", "sex", "age", "sibling/spouse", "parent/children"]

data = pandas.read_csv("./train.processed.csv")

results = tensor(data["Survived"].values, dtype=float32)
details = tensor(data.iloc[:, 2:7].values, dtype=float32)

details.shape

torch.Size([891, 5])

In [18]:
import torch
from torch import nn

device = "mps" if torch.backends.mps.is_available() else "cpu"

print("using device: {}".format(device))

class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 10),
            nn.ReLU(),
            nn.Linear(10, 2),
            nn.ReLU(),
            nn.Linear(2, 1)
        )

    def forward(self, x):
        logits = self.stack(x)
        return logits

# move data to gpu
results.to(device)
details.to(device)
model = NeuralNet().to(device)

print(model)

using device: mps
NeuralNet(
  (stack): Sequential(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=2, bias=True)
    (5): ReLU()
    (6): Linear(in_features=2, out_features=1, bias=True)
  )
)


In [27]:
x = torch.rand(5, device=device)
logits = model(x)

logits

tensor([0.4285], device='mps:0', grad_fn=<LinearBackward0>)

In [33]:
lr = 1e-3
batch_size = 32
epocs = 5
lossFn = nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [47]:
def train(input, label, model, loss_fn, optimizer):
    model.train()
    
    size = len(input)

    for index, case in enumerate(input):
        case = case.to(device)
        pred = model(case)
        correct = label[index].reshape(1)
        correct = correct.to(device)
        loss = loss_fn(pred, correct)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if index % 10 == 0:
            loss, current = loss.item(), index * len(case)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
train(details, results, model, lossFn, optimizer)

loss: 0.488635  [    0/  891]
loss: 0.944967  [   50/  891]
loss: 0.466464  [  100/  891]
loss: 0.482291  [  150/  891]
loss: 0.568158  [  200/  891]
loss: 0.533208  [  250/  891]
loss: 0.473202  [  300/  891]
loss: 0.478832  [  350/  891]
loss: 0.465933  [  400/  891]
loss: 0.474556  [  450/  891]
loss: 0.509157  [  500/  891]
loss: 0.460563  [  550/  891]
loss: 0.506610  [  600/  891]
loss: 0.418356  [  650/  891]
loss: 0.362833  [  700/  891]
loss: 0.404046  [  750/  891]
loss: 0.376982  [  800/  891]
loss: 0.394200  [  850/  891]
loss: 0.489955  [  900/  891]
loss: 1.062258  [  950/  891]
loss: 0.392167  [ 1000/  891]
loss: 0.399525  [ 1050/  891]
loss: 1.103586  [ 1100/  891]
loss: 0.928531  [ 1150/  891]
loss: 0.488090  [ 1200/  891]
loss: 0.390322  [ 1250/  891]
loss: 0.405791  [ 1300/  891]
loss: 0.423995  [ 1350/  891]
loss: 0.419297  [ 1400/  891]
loss: 0.944889  [ 1450/  891]
loss: 0.956516  [ 1500/  891]
loss: 0.893496  [ 1550/  891]
loss: 0.478592  [ 1600/  891]
loss: 0.76